Maven is the wonderful, confusing build tool for Java.

# Terminology
- When you run Maven, you can specify a goal or a phase.
- When executing Maven with a phase, it will pass/execute all phases up to and including the given phase.
- When executing Maven with a goal, it will pass/execute all phases up to the phase for that goal.
- Goals are executed in phases; Maven’s default lifecycle bindings show which goals will run in which phases.

# Properties of artifacts
- <optional>true</optional> on a dependency means that the dependency will not get pulled down (as transitive dependency) by default - unless, for example, the artifact is downloaded explicitly, or using dependency:go-offline

# Lifecycles and build phases
There are three built-in lifecycles:

- default
- clean
- site

# Standard build phases
The standard build phases for the default lifecycle are:

- validate
- initialize
- generate-sources - generate any source code for inclusion in compilation
- process-sources
- generate-resources - generate resources for inclusion in the package
- process-resources
- compile
- process-classes
- generate-test-sources
- process-test-sources
- generate-test-resources
- process-test-resources
- test-compile
- process-test-classes
- test
- prepare-package
- package
- pre-integration-test
- integration-test
- post-integration-test
- verify
- install
- deploy

# Lifecycle bindings

The packaging type dictates which goals are bound to each phase.
e.g. jar: binds jar:jar to the package phase, surefire:test to the test phase, etc.

# POMs

The Maven Super POM is located in:
```
lib/maven-model-builder-3.0.3.jar:org/apache/maven/model/pom-4.0.0.xml
```

# Repositories
Here are some standard repository IDs used by Maven:

- central (Maven Central) - which is defined in the Super POM. Refers to http://repo1.maven.org

# Installation
Software collections (packaged) installation on CentOS:

```sh
$ sudo yum install -y centos-release-scl
$ sudo yum install -y rh-maven35
$ scl enable rh-maven35 bash
```
Basic non-package installation on RHEL:
```
$ sudo mkdir -p /usr/share/maven /usr/share/maven/ref
$ sudo curl -fsSL -o /tmp/apache-maven.tar.gz http://mirror.ox.ac.uk/sites/rsync.apache.org/maven/maven-3/3.5.4/binaries/apache-maven-3.5.4-bin.tar.gz
$ sudo tar -zxf /tmp/apache-maven.tar.gz -C /usr/share/maven --strip-components=1
$ sudo rm -f /tmp/apache-maven.tar.gz
$ sudo ln -s /usr/share/maven/bin/mvn /usr/bin/mvn
```

# Cookbook
## Getting help
Getting detailed help on a plugin goal and parameters:

```
$ mvn help:describe -Dcmd=bundle:baseline-report -Ddetail
```
## Working with plugin goals

Execute a specific named execution:

```
$ mvn prefix:goal@your-execution-id
```

## Working offline
Get an artifact to a specific location:

```
$ mvn dependency:get \
    -Dartifact=org.arquillian.cube:arquillian-cube-openshift:1.9.0 \
    -Dmaven.repo.local=/home/tdonohue/tmp/arq190
```

Get an artifact and download all its dependencies offline (NB: This will also attempt to download any dependencies marked as optional=true)

```
$ mvn dependency:get -Dartifact=org.arquillian.cube:arquillian-cube-openshift:1.9.0
$ mvn -f /home/tdonohue/.m2/repository/org/arquillian/cube/arquillian-cube-openshift/1.9.0/arquillian-cube-openshift-1.9.0.pom \
    dependency:go-offline \
    -Dmaven.repo.local=/path/to/temporary/location
$ tar cvfz artifact-dependencies.tgz /path/to/temporary/location
```

Copy a project’s dependencies:

```
$ mvn dependency:copy-dependencies
```
Copy the bare minimum M2 repository needed to run a project:

```
$ rm -rf ~/.m2/repository
$ mvn dependency:go-offline
$ tar cvfz offline-repository.tgz ~/.m2/repository
```

## Deployment
Deploying to a repository:

```
$ mvn clean deploy -DaltDeploymentRepository=internalSnapshots::default::http://localhost:8081/nexus/content/repositories/snapshots/


$ mvn clean deploy -DaltDeploymentRepository=internalReleases::default::http://localhost:8081/nexus/content/repositories/releases/
```

## Debugging
To debug a Maven goal use the mvnDebug command - this starts a debug listener on port 8000:

```
$ mvnDebug <plugin:goal>
```

## Analysing dependencies
To find which dependency pulls in a particular transitive dependency:

```
$ mvn dependency:tree -Dincludes=org.apache.tomcat:tomcat-jdbc
...
[INFO] --- maven-dependency-plugin:2.8:tree (default-cli) @ my-application ---
[INFO] com.example:my-application:jar:1.0-SNAPSHOT
[INFO] \- org.springframework.boot:spring-boot-starter-jdbc:jar:1.5.16.RELEASE:compile
[INFO]    \- org.apache.tomcat:tomcat-jdbc:jar:8.0.36.redhat-30:compile
```

## Plugins
In a child POM, to override the configuration for a plugin which is defined on a parent, use the combine.self attribute, either on the configuration attribute or its children:

```
<configuration combine.self="override">

</configuration>
```

## Troubleshooting
When fetching an artifact from a local Maven repository:“Could not find artifact org.xx.x:xx-xx-xx:jar:2.23.2 in xxx” - but you know it should work.

Maven has probably already tried once to fetch the artifact. Re-run the build with Maven’s -U switch to force updates (try again).